# Benchmarking Energy Consumption of Frontend Frameworks

This notebook presents data analysis and visualization for a firefox profiler JSON file. We use the Firefox Profiler to capture performance data and Python’s PyAutoGUI library to automate user interactions. Please make sure to only keep the power metric of the target tab before saving the file.

Each profile contains `counters[]`:
- For the counter where `counter["description"] == "Power utilization"`:
   - `counters[0].samples.time[]` - Array of timestamps (in milliseconds) for each energy sample.
   - `counters[0].samples.count[]` - Array of corresponding energy measurements (in pWh).

- Energy at time[i] is count[i].
- Total energy consumed in the time interval [i...j] will be:
    $\sum_{k=i}^{j} count[k]$
- Power at time[i] is calculated as:
    
    $\frac{powerUsageInPwh \times 10^{-12} \times 1000 \times 3600}{time[i] - time[i-1]}$
    
    Where:
    - $powerUsageInPwh \times 10^{-12}$ converts pWh to Wh.
    - $\times 1000$ converts milliseconds to seconds.
    - $\times 3600$ converts seconds to hours.
    - Result will be in Watt.
 - NB: In my case because of the small units, I used mW instead of Watt, so I used 10e9 instead of 10e12.

## Install and import libraries

In [ ]:
%pip install pandas matplotlib numpy seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

## Helper Functions

In [ ]:
def load_profile_data(file_path):
    """
    Load JSON profile data from {file_path}.

    Args:
        file_path (str): Path to the JSON file.

    Returns:
        list: A list of dictionaries containing profile data.
    """
    try:
        with open(file_path, "r") as file:
            data = json.load(file)
            return data['counters']
    except FileNotFoundError:
        print(f"Warning: File {file_path} not found")

def extract_time_and_count(profile_data, PID):
    """
    Extract time and count values from the profile data.
    Normalizes time values to start from 0.
    
    Args:
        profile_data (list): List of dictionaries containing profile data.
        PID (str): Process ID to filter by.
        
    Returns:
        tuple: Two lists - normalized time values and count values.
    """
    time_values = []
    count_values = []
    power_counter = None
    
    for counter in profile_data:
        if counter["description"] == "Power utilization" and counter["pid"] == PID:
            power_counter = counter
            break
        
    if not power_counter:
        return [], []
    
    time_values = power_counter["samples"]["time"]
    count_values = power_counter["samples"]["count"]

    # These lines zero out the first values of the counters, as they are unreliable. In
    # addition, there are probably some missed counts in the memory counters, so the
    # first memory number slowly creeps up over time, and becomes very unrealistic.
    # In order to not be affected by these platform limitations, zero out the first
    # counter values.
    # "Memory counter in Gecko Profiler isn't cleared when starting a new capture"
    # https://bugzilla.mozilla.org/show_bug.cgi?id=1520587
    
    count_values[0] = 0
    
    # Normalize time values to start from 0
    if time_values:
        initial_time = time_values[0]
        time_values = [t - initial_time for t in time_values]
    
    return time_values, count_values

def calculate_power_consumption(time_values, count_values):
    """
    Calculate power consumption using the given time and count values.

    Args:
        time_values (list): List of time values in milliseconds.
        count_values (list): List of corresponding energy measurements in pWh.

    Returns:
        list: A list of power consumption values in Watts.
    """
    power_consumption = [0]
    for i in range(1, len(time_values)):
        time_diff = time_values[i] - time_values[i - 1]
        if time_diff > 0:
            power = (count_values[i] * SCALE * 1000 * 3600) / time_diff
            power_consumption.append(power)
    return power_consumption

def calculate_total_energy(df):
    return df["count"].sum() * SCALE

## Variables
- You can change the measurement units as needed.
- You will need to specify the `PID` of the browser tab. To find it, open the JSON profiler file using [Firefox Profiler](https://profiler.firefox.com/). The tab PID will be shown on the left side of the timeline view.
- Make sure to update the `PATH` variable to point to the JSON file you want to analyze.

In [ ]:
SCALE = 10**-9 #from pWh to mWh
POWER_UNIT = 'mW'
ENERGY_UNIT = 'mWh'
TIME_UNIT = 'ms'
PATH = "Firefox 2025-04-08 15.30 profile.json"

#Enter your PIDs
PID = "75775"

profile_data = load_profile_data(PATH)

times, counts = extract_time_and_count(profile_data, PID)


dataframe = pd.DataFrame({
    "time": times,
    "count": counts
})

# Compute power values
dataframe["power"] = calculate_power_consumption(dataframe["time"].tolist(), dataframe["count"].tolist())

# Calculate min, max, and avg power
min_power = dataframe["power"].min()
max_power = dataframe["power"].max()
avg_power = dataframe["power"].mean()

# Combined power graph on a single plot
plt.figure(figsize=(10, 6))
plt.plot(dataframe["time"], dataframe["power"], label="Tab", color="green")
plt.xlabel(f"Time ({TIME_UNIT})")
plt.ylabel(f"Power ({POWER_UNIT})")
plt.title("Power Consumption")
plt.legend()
plt.tight_layout()
#plt.savefig('1_power.png')
plt.show()

metrics = [f"Min Power ({POWER_UNIT})", f"Max Power ({POWER_UNIT})", f"Average Power ({POWER_UNIT})"]
values = [min_power, max_power, avg_power]
x = np.arange(len(metrics))
width = 0.35 
plt.figure(figsize=(10, 6))
plt.bar(x + width/2, values, width, label="Tab", color="green")
plt.xlabel("Metrics")
plt.ylabel("Values")
plt.title("Power Metrics")
plt.xticks(x, metrics, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

total_energy = calculate_total_energy(dataframe)

categories = [f"Total Energy ({ENERGY_UNIT})"]
values = [total_energy]
x = np.arange(len(categories))
width = 0.35

plt.figure(figsize=(6, 4))
bars = plt.bar(x + width/2, values, width, label="Tab", color="green")
 
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height, f'{total_energy:.6f} {ENERGY_UNIT}', ha='center', va='bottom')
 
plt.xlabel("Metrics")
plt.ylabel(f"Energy ({ENERGY_UNIT})")
plt.title("Total Energy Consumption")
plt.xticks(x, categories)
plt.legend()

plt.tight_layout()
#plt.savefig('1_energy2.png')

plt.show()